In [2]:
# 导入需要的库
import pandas as pd
import numpy as np

# 读取数据
df = pd.read_csv('customer_support_tickets.csv')

# 查看基本信息
print("数据形状:", df.shape)
print("\n前5行:")
print(df.head())


数据形状: (8469, 17)

前5行:
   Ticket ID        Customer Name              Customer Email  Customer Age  \
0          1        Marisa Obrien  carrollallison@example.com            32   
1          2         Jessica Rios    clarkeashley@example.com            42   
2          3  Christopher Robbins   gonzalestracy@example.com            48   
3          4     Christina Dillon    bradleyolson@example.org            27   
4          5    Alexander Carroll     bradleymark@example.com            67   

  Customer Gender Product Purchased Date of Purchase      Ticket Type  \
0           Other        GoPro Hero       2021-03-22  Technical issue   
1          Female       LG Smart TV       2021-05-22  Technical issue   
2           Other          Dell XPS       2020-07-14  Technical issue   
3          Female  Microsoft Office       2020-11-13  Billing inquiry   
4          Female  Autodesk AutoCAD       2020-02-04  Billing inquiry   

             Ticket Subject  \
0             Product setup   
1

In [3]:
# 查看所有列名
print("所有列名:")
print(df.columns.tolist())
print("\n" + "="*50 + "\n")

# 查看数据类型
print("数据类型:")
print(df.dtypes)
print("\n" + "="*50 + "\n")

# 查看缺失值
print("缺失值统计:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

# 看看Priority列（这应该是target variable）
if 'Ticket Priority' in df.columns:
    print("Priority分布:")
    print(df['Ticket Priority'].value_counts())
elif 'Priority' in df.columns:
    print("Priority分布:")
    print(df['Priority'].value_counts())

所有列名:
['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']


数据类型:
Ticket ID                         int64
Customer Name                    object
Customer Email                   object
Customer Age                      int64
Customer Gender                  object
Product Purchased                object
Date of Purchase                 object
Ticket Type                      object
Ticket Subject                   object
Ticket Description               object
Ticket Status                    object
Resolution                       object
Ticket Priority                  object
Ticket Channel                   object
First Response Time              object
Time to Resolution               object
Customer Satisfaction

In [4]:
# 查看Priority的类别
print("Ticket Priority分布:")
print(df['Ticket Priority'].value_counts())
print("\n占比:")
print(df['Ticket Priority'].value_counts(normalize=True))

Ticket Priority分布:
Ticket Priority
Medium      2192
Critical    2129
High        2085
Low         2063
Name: count, dtype: int64

占比:
Ticket Priority
Medium      0.258826
Critical    0.251387
High        0.246192
Low         0.243594
Name: proportion, dtype: float64


In [5]:
import re

# ============================================
# 1. 文本清洗函数
# ============================================
def clean_text(text):
    """
    清洗文本：转小写、去特殊字符、去多余空格
    """
    if pd.isna(text):  # 处理可能的NaN
        return ''
    
    # 转字符串（以防万一）
    text = str(text)
    
    # 转小写
    text = text.lower()
    
    # 去掉特殊字符，只保留字母、数字、空格
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    # 去掉多余空格
    text = ' '.join(text.split())
    
    return text

# ============================================
# 2. 应用清洗到Subject和Description
# ============================================
print("正在清洗文本...")

df['Ticket Subject_clean'] = df['Ticket Subject'].apply(clean_text)
df['Ticket Description_clean'] = df['Ticket Description'].apply(clean_text)

print("✓ 文本清洗完成")

# ============================================
# 3. 查看清洗效果
# ============================================
print("\n清洗前后对比（前3行）:")
print("\n--- 原始 Ticket Subject ---")
print(df['Ticket Subject'].head(3).tolist())
print("\n--- 清洗后 Ticket Subject ---")
print(df['Ticket Subject_clean'].head(3).tolist())

print("\n--- 原始 Ticket Description ---")
print(df['Ticket Description'].head(3).tolist())
print("\n--- 清洗后 Ticket Description ---")
print(df['Ticket Description_clean'].head(3).tolist())

正在清洗文本...
✓ 文本清洗完成

清洗前后对比（前3行）:

--- 原始 Ticket Subject ---
['Product setup', 'Peripheral compatibility', 'Network problem']

--- 清洗后 Ticket Subject ---
['product setup', 'peripheral compatibility', 'network problem']

--- 原始 Ticket Description ---
["I'm having an issue with the {product_purchased}. Please assist.\n\nYour billing zip code is: 71701.\n\nWe appreciate that you have requested a website address.\n\nPlease double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.", "I'm having an issue with the {product_purchased}. Please assist.\n\nIf you need to change an existing product.\n\nI'm having an issue with the {product_purchased}. Please assist.\n\nIf The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly.", "I'm facing a problem with my {product_purchased}. The {product_purchased} is not turning on. It was working fine until yesterday, but now it doesn't respond.\n\n1.

In [6]:
# ============================================
# 特征工程
# ============================================

print("正在创建特征...\n")

# 1. 合并Subject和Description
df['combined_text'] = df['Ticket Subject_clean'] + ' ' + df['Ticket Description_clean']

# 2. 创建文本长度特征
df['subject_length'] = df['Ticket Subject_clean'].str.len()
df['description_length'] = df['Ticket Description_clean'].str.len()
df['combined_length'] = df['combined_text'].str.len()

# 3. 创建词数特征
df['subject_word_count'] = df['Ticket Subject_clean'].str.split().str.len()
df['description_word_count'] = df['Ticket Description_clean'].str.split().str.len()
df['combined_word_count'] = df['combined_text'].str.split().str.len()

# 4. Ticket Type的频率编码（简单有效的categorical encoding）
ticket_type_freq = df['Ticket Type'].value_counts(normalize=True).to_dict()
df['ticket_type_frequency'] = df['Ticket Type'].map(ticket_type_freq)

print("✓ 特征创建完成\n")

# 查看新创建的特征
print("新特征预览:")
feature_cols = ['subject_length', 'description_length', 'combined_length', 
                'subject_word_count', 'description_word_count', 'combined_word_count',
                'ticket_type_frequency']
print(df[feature_cols].head())

# 统计信息
print("\n特征统计:")
print(df[feature_cols].describe())

正在创建特征...

✓ 特征创建完成

新特征预览:
   subject_length  description_length  combined_length  subject_word_count  \
0              13                 268              282                   2   
1              24                 262              287                   2   
2              15                 253              269                   2   
3              14                 248              263                   2   
4               9                 317              327                   2   

   description_word_count  combined_word_count  ticket_type_frequency  
0                      43                   45               0.206282  
1                      44                   46               0.206282  
2                      42                   44               0.206282  
3                      41                   43               0.192939  
4                      55                   57               0.192939  

特征统计:
       subject_length  description_length  combined_length  \
co

In [8]:
from sklearn.model_selection import train_test_split

# ============================================
# Train/Test Split
# ============================================

print("正在划分训练集和测试集...\n")

# 准备X和y
X = df[['combined_text', 'subject_length', 'description_length', 
        'combined_length', 'subject_word_count', 'description_word_count',
        'combined_word_count', 'ticket_type_frequency', 'Ticket Type']]

y = df['Ticket Priority']

# 80-20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # 保持priority分布一致
)

print(f"训练集大小: {len(X_train)} ({len(X_train)/len(df)*100:.1f}%)")
print(f"测试集大小: {len(X_test)} ({len(X_test)/len(df)*100:.1f}%)")

print("\n训练集Priority分布:")
print(y_train.value_counts(normalize=True))

print("\n测试集Priority分布:")
print(y_test.value_counts(normalize=True))

# ============================================
# 保存处理后的数据
# ============================================

print("\n正在保存文件...\n")

# 保存完整的清洗后数据
df.to_csv('tickets_cleaned_with_features.csv', index=False)
print("✓ 保存完整数据: tickets_cleaned_with_features.csv")

# 保存train/test split
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

print("✓ 保存训练数据: X_train.csv, y_train.csv")
print("✓ 保存测试数据: X_test.csv, y_test.csv")

print("\n" + "="*50)
print("数据清洗和特征工程完成！✨")
print("="*50)

正在划分训练集和测试集...

训练集大小: 6775 (80.0%)
测试集大小: 1694 (20.0%)

训练集Priority分布:
Ticket Priority
Medium      0.258893
Critical    0.251365
High        0.246199
Low         0.243542
Name: proportion, dtype: float64

测试集Priority分布:
Ticket Priority
Medium      0.258560
Critical    0.251476
High        0.246163
Low         0.243802
Name: proportion, dtype: float64

正在保存文件...

✓ 保存完整数据: tickets_cleaned_with_features.csv
✓ 保存训练数据: X_train.csv, y_train.csv
✓ 保存测试数据: X_test.csv, y_test.csv

数据清洗和特征工程完成！✨


In [13]:
# 简化版README
readme = """# IT Ticket Priority - Data Preprocessing

## Dataset
- Total: 8,469 tickets
- Classes: Critical, High, Medium, Low (balanced)

## What I Did
1. Cleaned text (lowercase, removed special chars)
2. Created features (text length, word count, combined text)
3. Split data 80-20 for train/test

## Output Files
- tickets_cleaned_with_features.csv (full data)
- X_train.csv, y_train.csv (training)
- X_test.csv, y_test.csv (testing)

## Next Steps for Team

Prepared by: Sara
"""

with open('README.txt', 'w') as f:
    f.write(readme)
    
print(readme)
print("\n✓ README saved as README.txt")

# IT Ticket Priority - Data Preprocessing

## Dataset
- Total: 8,469 tickets
- Classes: Critical, High, Medium, Low (balanced)

## What I Did
1. Cleaned text (lowercase, removed special chars)
2. Created features (text length, word count, combined text)
3. Split data 80-20 for train/test

## Output Files
- tickets_cleaned_with_features.csv (full data)
- X_train.csv, y_train.csv (training)
- X_test.csv, y_test.csv (testing)

## Next Steps for Team

Prepared by: Sara


✓ README saved as README.txt
